# User prompt
The user define the research objective a prompt. I use openAI to identify relevant key words for the research.

In [26]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load the API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Research objective
research_objective = """
At present, one of our clients is looking to speak with professionals who have insights about the emerging technologies 
in soft contact lens manufacturing, particularly non-injection moulded methods. They would broadly like to understand how 
these technologies are reshaping the industry—from on-demand manufacturing to smart, drug-delivery-enabled lenses.
"""

prompt = f"""
You are assisting a researcher in generating targeted search terms.

Return a JSON object with the following structure:
- "openalex": exactly 5 academic search terms, each consisting of a single word
- "patentview": exactly 5 patent-related keywords, each consisting of a single word
- "cpc_codes": exactly 5 relevant CPC classification codes

Instructions:
- All search terms must be single words (no spaces, no hyphens)
- Use only valid JSON
- Do not include markdown, comments, or explanations
- Ensure terms are concise, domain-relevant, and technically meaningful

Research Objective:
\"\"\"
{research_objective}
\"\"\"
"""




# OpenAI API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo", # Cost efficient model
    # model="gpt-4-turbo",
    messages=[{"role": "user", 
               "content": prompt}]
)

# Print the results
print(response.choices[0].message.content)

{
    "openalex": ["manufacturing", "technologies", "on-demand", "smart", "drug"],
    "patentview": ["lenses", "manufacturing", "technology", "innovation", "industry"],
    "cpc_codes": ["G02C7", "G02C7/04", "G02C7/08", "G02C7/10", "G02C7/12"]
}


# Openalex API
https://docs.openalex.org/

Valid parameters are: apc_sum, cited_by_count_sum, cursor, filter, format, group_by, group-by, group_bys, group-bys, mailto, page, per_page, per-page, q, sample, seed, search, select, sort, warm.'

In [ ]:
import requests
import pandas as pd

def reconstruct_abstract(abstract_inverted_index):
        
    # Variable to store the highest index
    max_value = 0
     # Loop through all the list of position in the abstract_inverted_index dictionary.
    for values in abstract_inverted_index.values():
        # Loop through all the index value
        for value in values:
            # identify the highest value index
            if value >= max_value:
                max_value = value
                
    # Create an empty list with abstract size        
    abstract = [None] * (max_value +1)

    # The length of the list should be one more than the highest number (position) found in the index.

    # Loop through each word in the abstract_inverted_index:
    for word, positions in abstract_inverted_index.items():
        # For each word, get the list of positions it appears in.
        for position in positions:
            # Insert each word into its correct position in the list.
            # assign the word to that position in the abstract list.
            abstract[position]= word
                        
    # Join all the words in the list into a single string, separated by spaces.
    abstract_text = " ".join(abstract)
    # print(abstract_text)
    
    return abstract_text

url = "https://api.openalex.org/works"

key_word = "manufacturing"
search_terms = f"contact lens AND {key_word}"
mailto = "adyl.elguamra@gmail.com" #For best performance, add your email to all API requests

per_page = 10 # By default there are 25 results per page
page = 1 # Get the result from page number

params = {
    "search": search_terms, # searches across titles, abstracts, and fulltext.
    "per_page": per_page, 
    "page": page, # if needed I can loop over pages. for example from page 1 to 5 with a for loop
    "sort": "relevance_score:desc",
    "mailto": mailto   
}
response = requests.get(url, params=params)

if response.status_code == 200:
    
    data = response.json()

    works = data.get("results", []) # access a key in dictionary
        
    # Extract info into list of dicts
    records = []
    # loop in the list
    for work in works:
        abstract_inverted_index = work.get("abstract_inverted_index", [])
        # print(abstract_inverted_index)
        abstract = reconstruct_abstract(abstract_inverted_index)

        record = {
            "Title": work.get("title"),
            "Abstract": abstract,
            "Year": work.get("publication_year"),
            "Citations": work.get("cited_by_count"),
            "Authors": [auth["author"]["display_name"] for auth in work.get("authorships", [])],
            "OpenAlex ID": work.get("id"),
        }
        records.append(record)

    print("Final URL:", response.url)
    print("JSON Data:", response.json())

    # Convert to DataFrame
    df = pd.DataFrame(records)
    display(df)

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

Final URL: https://api.openalex.org/works?search=contact+lens+AND+manufacturing&per_page=10&page=1&sort=relevance_score%3Adesc&mailto=adyl.elguamra%40gmail.com
JSON Data: {'meta': {'count': 108965, 'db_response_time_ms': 171, 'page': 1, 'per_page': 10, 'groups_count': None}, 'results': [{'id': 'https://openalex.org/W2044512869', 'doi': 'https://doi.org/10.1016/j.eurpolymj.2014.11.024', 'title': 'Biomedical applications of hydrogels: A review of patents and commercial products', 'display_name': 'Biomedical applications of hydrogels: A review of patents and commercial products', 'relevance_score': 469.79657, 'publication_year': 2014, 'publication_date': '2014-11-29', 'ids': {'openalex': 'https://openalex.org/W2044512869', 'doi': 'https://doi.org/10.1016/j.eurpolymj.2014.11.024', 'mag': '2044512869'}, 'language': 'en', 'primary_location': {'is_oa': True, 'landing_page_url': 'https://doi.org/10.1016/j.eurpolymj.2014.11.024', 'pdf_url': None, 'source': {'id': 'https://openalex.org/S13547235

,Title,Abstract,Year,Citations,Authors,OpenAlex ID
0,Biomedical applications of hydrogels: A review...,Hydrogels have become very popular due to thei...,2014,2285,"[Enrica Caló, Vitaliy V. Khutoryanskiy]",https://openalex.org/W2044512869
1,Multistate Outbreak of Fusarium Keratitis Asso...,Fusarium keratitis is a serious corneal infect...,2006,592,"[Douglas C. Chang, Gavin B. Grant, Kerry O’Don...",https://openalex.org/W2116582646
2,Prospects for Additive Manufacturing in Contac...,Additive manufacturing (3D printing) has the a...,2020,44,"[Fahad Alam, Mohamed Elsherif, Bader AlQattan,...",https://openalex.org/W3083896593
3,Contact Lens Sensors in Ocular Diagnostics,Contact lenses as a minimally invasive platfor...,2014,432,"[Nicholas M. Farandos, Ali K. Yetisen, Michael...",https://openalex.org/W2055793978
4,Contact Lens Materials: A Materials Science Pe...,More is demanded from ophthalmic treatments us...,2019,301,"[Christopher S. A. Musgrave, Fengzhou Fang]",https://openalex.org/W2909056211
5,Impact of manufacturing technology and materia...,Abstract Hydrogel materials are not only diffe...,2004,33,"[Carole Maldonado‐Codina, Nathan Efron]",https://openalex.org/W2113155567
6,Impact of manufacturing technology and materia...,Background: Our aim was to investigate the imp...,2005,33,"[Carole Maldonado‐Codina, Nathan Efron]",https://openalex.org/W2093492547
7,Contact Lens Technology: From Fundamentals to ...,Abstract Contact lenses are ocular prosthetic ...,2019,224,"[Rosalia Moreddu, Daniele Vigolo, Ali K. Yetisen]",https://openalex.org/W2949809649
8,A method of manufacturing microfluidic contact...,"In this paper, we present the development of m...",2018,24,"[Hongbin An, Liangzhou Chen, Xiaojun Liu, Bin ...",https://openalex.org/W2810586854
9,Contact lens practice,Part 1 Introduction Historical perspective. Th...,2002,169,[Nathan Efron],https://openalex.org/W1746299914


In [ ]:
if response.status_code == 200:
    data = response.json()
    works = data.get("results", [])
    
    # Extract info into list of dicts
    records = []
    for work in works:
        record = {
            "Title": work.get("title"),
            "Year": work.get("publication_year"),
            "Citations": work.get("cited_by_count"),
            "Authors": ", ".join([auth["author"]["display_name"] for auth in work.get("authorships", [])]),
            "OpenAlex ID": work.get("id"),
        }
        records.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(records)
    display(df)

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)
